Imports

In [76]:
import numpy as np
from classiq import *
from classiq.qmod.symbolic import logical_and, pi
from classiq.execution import (
    ClassiqBackendPreferences,
    ClassiqSimulatorBackendNames,
    ExecutionPreferences,
)

Initializing Simulation Data for Coupled Harmonic Oscillators

In [52]:
# Useful Matrices and constants

# Number of oscillators = 2^n (power of 2)
n = 2

# Defines the error = e^(-r)
r = 4 # Roughly an error of 0.02

# Mass matrix
M = np.diag([1]*n)

# F Matrix (matrix of coefficients)
K = np.ones((n, n))
F = np.diag([n+1]*n) - K

# Displacements
x = np.zeros(n)

# Velocities
x_dot = np.zeros(n)

# Initial conditions setup
x_dot[0] = 1
x_dot[1] = -1

m_max = np.max(M.diagonal())
m_min = np.min(M.diagonal())
k_max = np.max(K)

Aleph = k_max / m_min



# IMPLEMENTING B DAGGER

The implementation of the B Dagger operator as per the description provided in the paper in Appendix A part 3. It follows the specific steps by developing each operation individually and then combining them in the end to apply the operator.

In [53]:
# SWAP 2 given registers of the same size

@qfunc 
def bitwise_swap(a: QArray[QBit], b:QArray[QBit]):
    """ Apply the bitwise swap for 2 given qubit registers a and b"""
    repeat(count= a.len,
           iteration= lambda i: SWAP(a[i],b[i]))

The S Oracle as described in Appendix A part 1

In [71]:
# S Operator

# Compute masses
def get_mass_bit_repr(index: int):
    fractional_M = M / m_max

    proportional_mass = np.floor(fractional_M[index][index] * (2**r))

    return proportional_mass if proportional_mass < 2**r else 2**r - 1
    

# Oracle to make mass
@qfunc
def S_m(j: QNum[n, False, 0], m_j: QNum[r, False, 0]):
    
    for i in range(n):
        control(j == i, lambda: inplace_prepare_int(get_mass_bit_repr(i), m_j))
        

# Compute spring constants
def get_k_bit_repr(j: int, k:int):
    fractional_K = K / k_max

    proportional_k = np.floor(fractional_K[j][k] * (2**r))

    return proportional_k if proportional_k < 2**r else 2**r - 1

# Oracle to make spring constants
@qfunc
def S_k(j: QNum[n, False, 0], k: QNum[n, False, 0], k_jk: QNum[r, False, 0]):
    for i in range(n):
        for l in range(n):
            control(logical_and(j == i, k == l), lambda: inplace_prepare_int(get_k_bit_repr(i, l), k_jk))
    

Arithmetic Inequality Testing as per step 5 and 7 in Appendix A part 2 and 3

In [55]:
# Arithmetic Inequality Testing (step 5) ->  k_jk < x^2 * m_j * N

@qfunc
def inequality_test_to_induce_amplitude(k_jk: QNum[r, False, 0], m_j: QNum[r, False, 0], target: QBit):
    x = QNum("x", r, False, r)

    allocate(r, x)
    
    hadamard_transform(x)
    
    control((k_jk * k_max / 2**r) <= ((m_max * m_j / 2**r) * x**2 * Aleph), lambda: X(target))

    hadamard_transform(x)

In [56]:
# Inequality Testing to rearrange sum (step 7)
@qfunc
def apply_test(j: QNum[n, False, 0], k: QNum[n, False, 0], target: QBit):
    bitwise_swap(j, k)
    X(target)


@qfunc
def inequality_test_to_rearrange_sum(j: QNum[n, False, 0], k: QNum[n, False, 0], target: QBit):
    control(k < j, lambda: apply_test(j, k, target))

Complete implementation of B Dagger including all steps outlined in Appendix A (till part 3)

In [57]:
# IMPLEMENTATION OF B^

@qfunc
def B_dagger(j: QNum[n, False, 0], k: QNum[n, False, 0]):
    m_j = QNum("m_j", r, False, 0)
    k_jk = QNum("k_jk", r, False, 0)

    allocate(r, m_j)
    allocate(r, k_jk)

    # Step 3, 6
    def compute_values():
        S_m(j, m_j)
        S_k(j, k, k_jk)

    # Step 4, 5
    target1 = QBit("target1")
    allocate(1, target1)

    within_apply(
        lambda: compute_values(),
        lambda: inequality_test_to_induce_amplitude(k_jk, m_j, target1)
    )

    # Step 7
    target2 = QBit("target2")
    allocate(1, target2)

    inequality_test_to_rearrange_sum(j, k, target2)

    # Step 8
    Z(target2)
    H(target2)
    

# HAMILTONIAN ENCODING

The Hamiltonian encoding is done with the help of controlled operations of the B_dagger and B unitary matrices and a few other simple operations such as the unitary projection and X gates as outlined in Appendix A part 4 of the paper. Following is the implementation of U_cond that is used in the hamiltonian encoding

In [59]:
@qfunc
def U_cond(register: QArray[QBit]):
    ancilla = QBit("cond_ancilla")
    allocate(1, ancilla)

    def conditions():
        control(ancilla == 0, lambda: reflect_about_zero(register))
        control(ancilla == 1, lambda: IDENTITY(register))
    
    within_apply(
        lambda: H(ancilla),
        lambda: conditions()
    )

The hamiltonian encoding is then done with the help of the previously developed operators through block encoding the B and B_dagger operators

In [60]:
@qfunc
def Hamiltonian(register: QArray[QBit], proj: QArray[QBit]):

    k = QNum("k", n, False, 0)
    allocate(n, k)
    hadamard_transform(k)
    
    ctrl = QBit("ctrl")
    allocate(1, ctrl)
    

    def apply_B():
        invert(lambda: B_dagger(register, k))
        

    def apply_B_dagger():
        B_dagger(register, k)
        

    def apply_controlled_unitaries():
        control(ctrl == 1, lambda: apply_B())
        U_cond(proj)
        control(ctrl == 0, lambda: apply_B_dagger())

        X(ctrl)
        

    within_apply(
        lambda: H(ctrl),
        lambda: apply_controlled_unitaries()
    )
    

# TIME EVOLUTION (using qsvt)

Most time evolution techniques involving QSP and Qubitization for this specific hamiltonian encoding provide similar runtimes and efficiency. Thus for this case, QSVT has been chosen to simulate the time evolution of the hamiltionian.

In [39]:
EVOLUTION_TIME = 10
EPS = 0.1

normalization_factor = np.sqrt(2 * Aleph * n)

normalized_time = normalization_factor * EVOLUTION_TIME

In [40]:
def get_normalized_lcu_coef(lcu_coef):

    normalization_factor = sum(lcu_coef)
    prepare_prob = [c / normalization_factor for c in lcu_coef]
    coef_size = int(np.ceil(np.log2(len(prepare_prob))))
    prepare_prob += [0] * (2**coef_size - len(prepare_prob))

    print("The size of the block encoding:", coef_size)
    print("The normalized coefficients:", prepare_prob)
    print("The normalization factor:", normalization_factor)

    return normalization_factor, coef_size, prepare_prob

In [41]:
@qfunc
def apply_pauli_term(pauli_string: PauliTerm, x: QArray[QBit]):
    repeat(
        count=x.len,
        iteration=lambda index: switch(
            pauli_string.pauli[index],
            [
                lambda: IDENTITY(x[pauli_string.pauli.len - index - 1]),
                lambda: X(x[pauli_string.pauli.len - index - 1]),
                lambda: Y(x[pauli_string.pauli.len - index - 1]),
                lambda: Z(x[pauli_string.pauli.len - index - 1]),
            ],
        ),
    )


@qfunc
def lcu_paulis(
    pauli_terms_list: CArray[PauliTerm],
    probs: CArray[CReal],
    block: QNum,
    data: QArray[QBit],
):
    within_apply(
        lambda: inplace_prepare_state(probs, 0.0, block),
        lambda: repeat(
            count=pauli_terms_list.len,
            iteration=lambda i: control(
                block == i, lambda: apply_pauli_term(pauli_terms_list[i], data)
            ),
        ),
    )

In [42]:
import pyqsp
from pyqsp.angle_sequence import Polynomial, QuantumSignalProcessingPhases

pg_cos = pyqsp.poly.PolyCosineTX()
pcoefs_cos, scale_cos = pg_cos.generate(
    epsilon=EPS, tau=normalized_time, return_scale=True
)
poly_cos = Polynomial(pcoefs_cos)

pg_sin = pyqsp.poly.PolySineTX()
pcoefs_sin, scale_sin = pg_sin.generate(
    epsilon=EPS, tau=normalized_time, return_scale=True
)
poly_sin = Polynomial(pcoefs_sin)

29.18991978446863
R=14
[PolyCosineTX] rescaling by 0.5.
29.18991978446863
R=14
[PolySineTX] rescaling by 0.5.


In [43]:
ang_seq_cos = QuantumSignalProcessingPhases(
    poly_cos,
    signal_operator="Wx",
    method="laurent",
    measurement="x",
    tolerance=0.00001,  # relaxing the tolerance to get convergence
)

ang_seq_sin = QuantumSignalProcessingPhases(
    poly_sin,
    signal_operator="Wx",
    method="laurent",
    measurement="x",
    tolerance=0.0001,  # relaxing the tolerance to get convergence
)


# change W(x) to R(x), as the phases are in the W(x) conventions see Eq.()
def convert_phases_to_Rx_convention(ang_seq):

    phases = np.array(ang_seq)
    phases[1:-1] = phases[1:-1] - np.pi / 2
    phases[0] = phases[0] - np.pi / 4
    phases[-1] = phases[-1] + (2 * (len(phases) - 1) - 1) * np.pi / 4

    # verify conventions. minus is due to exp(-i*phi*z) in qsvt in comparison to qsp
    phases = -2 * phases

    return phases.tolist()

In [44]:
phases_cos = convert_phases_to_Rx_convention(ang_seq_cos)
phases_sin = convert_phases_to_Rx_convention(ang_seq_sin)

In [45]:
@qfunc
def identify_block(num_data_qubits: CInt, qvar: QArray[QBit], qubit: QBit):
    block_qubits = QNum("block_qubits", qvar.len - num_data_qubits, False, 0)
    data = QArray("data", length=num_data_qubits)

    bind(qvar, [block_qubits, data])
    qubit ^= block_qubits == 0
    bind([block_qubits, data], qvar)


# defining a qsvt for the specific example
@qfunc
def my_qsvt(phases: CArray[CReal], qsvt_aux: QBit, block_ham:QArray, data: QArray):
    combined_vars = QArray("combined_vars")
    bind([block_ham, data], combined_vars)

    qsvt(
        phase_seq=phases,
        proj_cnot_1=lambda qvar, qubit: identify_block(data.len, qvar, qubit),
        proj_cnot_2=lambda qvar, qubit: identify_block(data.len, qvar, qubit),
        u=lambda x: Hamiltonian(x[block_ham.len:x.len], x[0:block_ham.len]),
        qvar=combined_vars,
        aux=qsvt_aux,
    )

    bind(combined_vars, [block_ham, data])

In [46]:
@qfunc
def main(
    qsvt_aux: Output[QBit],
    block_ham: Output[QNum],
    data: Output[QNum],
    block_exp: Output[QBit],
):
    allocate(1, qsvt_aux)
    allocate(1, block_exp)
    allocate(n, block_ham)
    prepare_amplitudes([1/np.sqrt(2**n)]*2**n, 0.0, data)
    within_apply(
        lambda: H(block_exp),
        lambda: (
            control(
                block_exp == 0,  # cosine
                lambda: my_qsvt(phases_cos, qsvt_aux, block_ham, data),
            ),
            control(
                block_exp == 1,  # sine
                lambda: (
                    U(0, 0, 0, pi / 2, qsvt_aux),  # for the i factor
                    my_qsvt(phases_sin, qsvt_aux, block_ham, data),
                ),
            ),
        ),
    )

In [48]:
quantum_model = create_model(main)
write_qmod(quantum_model, "hamiltonian_simulation_qsvt", decimal_precision=12)

In [ ]:
quantum_prog = synthesize(quantum_model)
show(qauntum_prog)

# Quantum Error Correction using Surface Codes

Further increasing accuracy of the quantum program through applying quantum error correction techniques involving the use of surface codes

In [ ]:
import numpy as np
from classiq import *
from classiq.qmod.symbolic import logical_and

# Define the number of qubits for the surface code
n = 5  # Adjust based on your needs

# Surface Code Error Correction Functions
@qfunc
def initialize_surface_code(qubits: QArray[QBit]):
    """Initialize the surface code lattice."""
    # Initialize all qubits to the |0> state
    X(qubits)

@qfunc
def measure_stabilizers(qubits: QArray[QBit], ancillas: QArray[QBit]):
    """Measure the stabilizers of the surface code."""
    # Measure X-type stabilizers
    for i in range(0, len(qubits), 2):
        CNOT(qubits[i], ancillas[i // 2])
        CNOT(qubits[i + 1], ancillas[i // 2])
        H(ancillas[i // 2])
        measure(ancillas[i // 2])

    # Measure Z-type stabilizers
    for i in range(1, len(qubits), 2):
        CNOT(qubits[i], ancillas[i // 2])
        CNOT(qubits[i + 1], ancillas[i // 2])
        measure(ancillas[i // 2])

@qfunc
def correct_errors(qubits: QArray[QBit], ancillas: QArray[QBit]):
    """Correct errors based on stabilizer measurements."""
    # Correct X errors
    for i in range(len(ancillas)):
        control(ancillas[i] == 1, lambda: X(qubits[i * 2]))

    # Correct Z errors
    for i in range(len(ancillas)):
        control(ancillas[i] == 1, lambda: Z(qubits[i * 2 + 1]))

@qfunc
def surface_code_qec(qubits: QArray[QBit], ancillas: QArray[QBit]):
    """Perform a full cycle of surface code error correction."""
    measure_stabilizers(qubits, ancillas)
    correct_errors(qubits, ancillas)

@qfunc
def main(j: Output[QNum]):
    # Allocate qubits for the surface code
    data_qubits = QArray("data_qubits", n)
    ancilla_qubits = QArray("ancilla_qubits", n // 2)

    # Initialize the surface code
    initialize_surface_code(data_qubits)

    # Allocate and prepare the Hamiltonian simulation
    allocate(n, j)
    hadamard_transform(j)

    # Time evolution with error correction
    for _ in range(10):  # Number of time steps
        # Insert your time evolution function here
        # time_evolution(j)

        # Perform error correction
        surface_code_qec(data_qubits, ancilla_qubits)

    # Finalize and measure
    measure(data_qubits)

# Synthesize and execute the quantum program
quantum_program = synthesize(create_model(main))
show(quantum_program)